In [2]:
import os, shutil

# 存放小数据集的目录
base_dir = 'F:/Jupyter/2019.6/weldingline/colordata'

# 建立训练集、验证集、测试集目录
train_dir = os.path.join(base_dir, 'train')
# os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
# os.mkdir(validation_dir)
#test_dir = os.path.join(base_dir, 'test')
# os.mkdir(test_dir)
print(train_dir,validation_dir)
# 将正负样本按照训练、验证、测试分类
train_pos_dir = os.path.join(train_dir, 'pos')
train_asymmetry_dir = os.path.join(train_dir, 'asymmetry')
train_overlap_dir = os.path.join(train_dir, 'overlap')
train_pore_dir = os.path.join(train_dir, 'pore')
train_splash_dir = os.path.join(train_dir, 'splash')
train_stain_dir = os.path.join(train_dir, 'stain')


validation_pos_dir = os.path.join(validation_dir, 'pos')
validation_asymmetry_dir = os.path.join(validation_dir, 'asymmetry')
validation_overlap_dir = os.path.join(validation_dir, 'overlap')
validation_pore_dir = os.path.join(validation_dir, 'pore')
validation_splash_dir = os.path.join(validation_dir, 'splash')
validation_stain_dir = os.path.join(validation_dir, 'stain')

pos_num =  len(os.listdir(train_pos_dir))
asymmetry_num =  len(os.listdir(train_asymmetry_dir))
overlap_num =  len(os.listdir(train_overlap_dir))
pore_num =  len(os.listdir(train_pore_dir))
splash_num =  len(os.listdir(train_splash_dir))
stain_num =  len(os.listdir(train_stain_dir))
total_num = pos_num+asymmetry_num+overlap_num+pore_num+splash_num+stain_num
print('total train_pos_img is ', pos_num)
print('total train_asymmetry_img is ', asymmetry_num)
print('total train_overlap_img is ', overlap_num)
print('total train_pore_img is ',pore_num)
print('total train_splash_img is ',splash_num)
print('total train_stain_img is ',stain_num)
print('total train is ', total_num)

vpos_num = len(os.listdir(validation_pos_dir))
vasymmetry_num = len(os.listdir(validation_asymmetry_dir))
voverlap_num = len(os.listdir(validation_overlap_dir))
vpore_num = len(os.listdir(validation_pore_dir))
vsplash_num = len(os.listdir(validation_splash_dir))
vstain_num = len(os.listdir(validation_stain_dir))
vtotal_num = vpos_num+vpore_num+voverlap_num+vsplash_num+vstain_num

print('total validation_pos_img is ',vpos_num )
print('total validation_asymmetry_img is ', vasymmetry_num)
print('total validation_overlap_img is ', voverlap_num)
print('total validation_pore_img is ', vpore_num)
print('total validation_splash_img is ', vsplash_num)
print('total validation_stain_img is ', vstain_num)
print('total validation is ', vtotal_num)


F:/Jupyter/2019.6/weldingline/colordata\train F:/Jupyter/2019.6/weldingline/colordata\validation
total train_pos_img is  243
total train_asymmetry_img is  102
total train_overlap_img is  53
total train_pore_img is  92
total train_splash_img is  22
total train_stain_img is  214
total train is  726
total validation_pos_img is  122
total validation_asymmetry_img is  35
total validation_overlap_img is  16
total validation_pore_img is  21
total validation_splash_img is  7
total validation_stain_img is  45
total validation is  211


In [6]:
import tensorflow as tf
import numpy as np
from keras import layers
from keras import models

# 搭建一个模型
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(512, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 510, 254, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 255, 127, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 253, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 126, 62, 64)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 124, 60, 128)      73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 30, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 60, 28, 128)       147584    
__________

In [ ]:
from keras import optimizers

#配置模型
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# 使用 ImageDataGenerator 从目录中读取图像
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255) # 将所有图像乘以 1/255 缩放
train_generator = train_datagen.flow_from_directory( 
        train_dir, #目标目录
        target_size=(512, 256),  #将所有图像的大小调整为 512×384
        batch_size=25, 
        class_mode='categorical'
        #classes = ['pos','asymmetry'.'overlap','pore','splash','stain']
) 

validation_generator = test_datagen.flow_from_directory( 
        validation_dir,
        target_size=(512, 256),
        batch_size=20,
        class_mode='categorical')

# 利用批量生成器拟合模型
history = model.fit_generator(
    train_generator,
    steps_per_epoch=29,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=26)


Found 716 images belonging to 6 classes.
Found 246 images belonging to 6 classes.
Epoch 1/20
29/29 [==============================] - 173s 6s/step - loss: 1.6220 - acc: 0.3350 - val_loss: 1.4240 - val_acc: 0.5122
Epoch 2/20
29/29 [==============================] - 172s 6s/step - loss: 1.4120 - acc: 0.4504 - val_loss: 1.2693 - val_acc: 0.4919
Epoch 3/20
10/29 [=========>....................] - ETA: 1:39 - loss: 1.2760 - acc: 0.5200

In [ ]:
#save
#model.save('noise.h5')

In [ ]:
"""Found 452 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/30
12/12 [==============================] - 63s 5s/step - loss: 0.8853 - acc: 0.5042 - val_loss: 0.6536 - val_acc: 0.6300
Epoch 2/30
12/12 [==============================] - 49s 4s/step - loss: 0.6981 - acc: 0.5276 - val_loss: 0.6234 - val_acc: 0.6300
Epoch 3/30
12/12 [==============================] - 57s 5s/step - loss: 0.6385 - acc: 0.6500 - val_loss: 0.5252 - val_acc: 0.7700
Epoch 4/30
12/12 [==============================] - 52s 4s/step - loss: 0.5776 - acc: 0.6941 - val_loss: 0.6826 - val_acc: 0.6200
Epoch 5/30
12/12 [==============================] - 53s 4s/step - loss: 0.5325 - acc: 0.7042 - val_loss: 0.5150 - val_acc: 0.7300
Epoch 6/30
12/12 [==============================] - 53s 4s/step - loss: 0.5350 - acc: 0.7240 - val_loss: 0.4076 - val_acc: 0.8600
Epoch 7/30
12/12 [==============================] - 58s 5s/step - loss: 0.4659 - acc: 0.7708 - val_loss: 0.4559 - val_acc: 0.7900
Epoch 8/30
12/12 [==============================] - 60s 5s/step - loss: 0.5168 - acc: 0.7333 - val_loss: 0.4138 - val_acc: 0.8300
Epoch 9/30
12/12 [==============================] - 54s 5s/step - loss: 0.4373 - acc: 0.8000 - val_loss: 0.3505 - val_acc: 0.8500
Epoch 10/30
12/12 [==============================] - 52s 4s/step - loss: 0.4342 - acc: 0.7500 - val_loss: 0.7959 - val_acc: 0.6200
Epoch 11/30
12/12 [==============================] - 50s 4s/step - loss: 0.3912 - acc: 0.8083 - val_loss: 0.4049 - val_acc: 0.7900
Epoch 12/30
12/12 [==============================] - 56s 5s/step - loss: 0.3935 - acc: 0.8136 - val_loss: 0.3147 - val_acc: 0.8400
Epoch 13/30
12/12 [==============================] - 62s 5s/step - loss: 0.4115 - acc: 0.8125 - val_loss: 0.2834 - val_acc: 0.9100
Epoch 14/30
12/12 [==============================] - 59s 5s/step - loss: 0.3228 - acc: 0.8554 - val_loss: 0.7848 - val_acc: 0.6600
Epoch 15/30
12/12 [==============================] - 59s 5s/step - loss: 0.3739 - acc: 0.8333 - val_loss: 0.3334 - val_acc: 0.8500
Epoch 16/30
12/12 [==============================] - 55s 5s/step - loss: 0.3055 - acc: 0.8805 - val_loss: 0.2862 - val_acc: 0.9100
Epoch 17/30
12/12 [==============================] - 61s 5s/step - loss: 0.3475 - acc: 0.8458 - val_loss: 0.3223 - val_acc: 0.8400
Epoch 18/30
12/12 [==============================] - 63s 5s/step - loss: 0.3104 - acc: 0.8872 - val_loss: 0.2798 - val_acc: 0.9100
Epoch 19/30
12/12 [==============================] - 64s 5s/step - loss: 0.3511 - acc: 0.8375 - val_loss: 0.2848 - val_acc: 0.8800
Epoch 20/30
12/12 [==============================] - 60s 5s/step - loss: 0.2657 - acc: 0.8904 - val_loss: 0.2947 - val_acc: 0.8600
Epoch 21/30
12/12 [==============================] - 60s 5s/step - loss: 0.2568 - acc: 0.8833 - val_loss: 0.3533 - val_acc: 0.8300
Epoch 22/30
12/12 [==============================] - 57s 5s/step - loss: 0.3080 - acc: 0.9046 - val_loss: 0.2715 - val_acc: 0.8900
Epoch 23/30
12/12 [==============================] - 57s 5s/step - loss: 0.2268 - acc: 0.8904 - val_loss: 0.3123 - val_acc: 0.8400
Epoch 24/30
12/12 [==============================] - 57s 5s/step - loss: 0.2115 - acc: 0.9167 - val_loss: 0.2552 - val_acc: 0.9000
Epoch 25/30
12/12 [==============================] - 60s 5s/step - loss: 0.2084 - acc: 0.9222 - val_loss: 0.3008 - val_acc: 0.8600
Epoch 26/30
12/12 [==============================] - 67s 6s/step - loss: 0.2252 - acc: 0.9042 - val_loss: 0.2770 - val_acc: 0.9000
Epoch 27/30
12/12 [==============================] - 60s 5s/step - loss: 0.1156 - acc: 0.9698 - val_loss: 0.3983 - val_acc: 0.8300
Epoch 28/30
12/12 [==============================] - 57s 5s/step - loss: 0.2255 - acc: 0.9125 - val_loss: 0.2761 - val_acc: 0.8800
Epoch 29/30
12/12 [==============================] - 61s 5s/step - loss: 0.0700 - acc: 0.9916 - val_loss: 0.5414 - val_acc: 0.7700
Epoch 30/30
12/12 [==============================] - 66s 6s/step - loss: 0.2182 - acc: 0.9208 - val_loss: 0.2619 - val_acc: 0.8900"""

In [ ]:
"""Found 904 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/30
30/30 [==============================] - 231s 8s/step - loss: 0.7069 - acc: 0.6111 - val_loss: 0.6436 - val_acc: 0.6450
Epoch 2/30
30/30 [==============================] - 305s 10s/step - loss: 0.5649 - acc: 0.7091 - val_loss: 0.4161 - val_acc: 0.8550
Epoch 3/30
30/30 [==============================] - 266s 9s/step - loss: 0.4837 - acc: 0.7553 - val_loss: 0.3666 - val_acc: 0.8650
Epoch 4/30
30/30 [==============================] - 199s 7s/step - loss: 0.4222 - acc: 0.7939 - val_loss: 0.3551 - val_acc: 0.8450
Epoch 5/30
30/30 [==============================] - 244s 8s/step - loss: 0.3607 - acc: 0.8473 - val_loss: 0.2984 - val_acc: 0.8550
Epoch 6/30
30/30 [==============================] - 226s 8s/step - loss: 0.3388 - acc: 0.8540 - val_loss: 0.2475 - val_acc: 0.9150
Epoch 7/30
30/30 [==============================] - 254s 8s/step - loss: 0.3039 - acc: 0.8874 - val_loss: 0.2316 - val_acc: 0.9350
Epoch 8/30
30/30 [==============================] - 225s 8s/step - loss: 0.2629 - acc: 0.8988 - val_loss: 0.2371 - val_acc: 0.9200
Epoch 9/30
30/30 [==============================] - 252s 8s/step - loss: 0.2346 - acc: 0.9133 - val_loss: 0.3019 - val_acc: 0.8550
Epoch 10/30
30/30 [==============================] - 285s 10s/step - loss: 0.2017 - acc: 0.9188 - val_loss: 0.2092 - val_acc: 0.9150
Epoch 11/30
30/30 [==============================] - 227s 8s/step - loss: 0.1829 - acc: 0.9171 - val_loss: 0.3318 - val_acc: 0.8500
Epoch 12/30
30/30 [==============================] - 186s 6s/step - loss: 0.1160 - acc: 0.9611 - val_loss: 0.2074 - val_acc: 0.8950
Epoch 13/30
30/30 [==============================] - 197s 7s/step - loss: 0.1399 - acc: 0.9466 - val_loss: 0.1960 - val_acc: 0.9550
Epoch 14/30
30/30 [==============================] - 265s 9s/step - loss: 0.0854 - acc: 0.9733 - val_loss: 0.5439 - val_acc: 0.8050
Epoch 15/30
30/30 [==============================] - 244s 8s/step - loss: 0.0750 - acc: 0.9789 - val_loss: 0.2478 - val_acc: 0.9200
Epoch 16/30
30/30 [==============================] - 254s 8s/step - loss: 0.0728 - acc: 0.9778 - val_loss: 0.2850 - val_acc: 0.8750
Epoch 17/30
30/30 [==============================] - 204s 7s/step - loss: 0.0701 - acc: 0.9800 - val_loss: 0.1441 - val_acc: 0.9600
Epoch 18/30
30/30 [==============================] - 235s 8s/step - loss: 0.0985 - acc: 0.9630 - val_loss: 0.1829 - val_acc: 0.9350
Epoch 19/30
30/30 [==============================] - 348s 12s/step - loss: 0.0793 - acc: 0.9766 - val_loss: 0.1604 - val_acc: 0.9450
Epoch 20/30
30/30 [==============================] - 325s 11s/step - loss: 0.0349 - acc: 0.9933 - val_loss: 0.1624 - val_acc: 0.9450
Epoch 21/30
30/30 [==============================] - 326s 11s/step - loss: 0.0057 - acc: 1.0000 - val_loss: 0.2513 - val_acc: 0.9350
Epoch 22/30
30/30 [==============================] - 279s 9s/step - loss: 0.0615 - acc: 0.9800 - val_loss: 0.1466 - val_acc: 0.9350
Epoch 23/30
30/30 [==============================] - 235s 8s/step - loss: 0.0198 - acc: 0.9933 - val_loss: 0.2007 - val_acc: 0.9450
Epoch 24/30
30/30 [==============================] - 195s 6s/step - loss: 0.0531 - acc: 0.9855 - val_loss: 0.1417 - val_acc: 0.9500
Epoch 25/30
30/30 [==============================] - 212s 7s/step - loss: 0.0042 - acc: 1.0000 - val_loss: 0.1527 - val_acc: 0.9600
Epoch 26/30
30/30 [==============================] - 292s 10s/step - loss: 0.0780 - acc: 0.9811 - val_loss: 0.1450 - val_acc: 0.9550
Epoch 27/30
30/30 [==============================] - 326s 11s/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.1521 - val_acc: 0.9400
Epoch 28/30
30/30 [==============================] - 240s 8s/step - loss: 0.0198 - acc: 0.9933 - val_loss: 0.1235 - val_acc: 0.9350
Epoch 29/30
30/30 [==============================] - 227s 8s/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.1463 - val_acc: 0.9500
Epoch 30/30
30/30 [==============================] - 230s 8s/step - loss: 0.0546 - acc: 0.9889 - val_loss: 0.1647 - val_acc: 0.9400"""

In [ ]:
"""Found 904 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/30
12/12 [==============================] - 71s 6s/step - loss: 0.8666 - acc: 0.5583 - val_loss: 0.6512 - val_acc: 0.6100
Epoch 2/30
12/12 [==============================] - 58s 5s/step - loss: 0.7146 - acc: 0.5292 - val_loss: 0.6806 - val_acc: 0.3900
Epoch 3/30
12/12 [==============================] - 56s 5s/step - loss: 0.6583 - acc: 0.6333 - val_loss: 0.5295 - val_acc: 0.7700
Epoch 4/30
12/12 [==============================] - 53s 4s/step - loss: 0.6553 - acc: 0.6173 - val_loss: 0.5532 - val_acc: 0.7800
Epoch 5/30
12/12 [==============================] - 53s 4s/step - loss: 0.5060 - acc: 0.7458 - val_loss: 1.3315 - val_acc: 0.3900
Epoch 6/30
12/12 [==============================] - 55s 5s/step - loss: 0.6505 - acc: 0.6917 - val_loss: 0.4900 - val_acc: 0.7750
Epoch 7/30
12/12 [==============================] - 55s 5s/step - loss: 0.5663 - acc: 0.7333 - val_loss: 0.4555 - val_acc: 0.8150
Epoch 8/30
12/12 [==============================] - 58s 5s/step - loss: 0.4393 - acc: 0.8115 - val_loss: 0.4947 - val_acc: 0.7750
Epoch 9/30
12/12 [==============================] - 57s 5s/step - loss: 0.5273 - acc: 0.7458 - val_loss: 0.4592 - val_acc: 0.7900
Epoch 10/30
12/12 [==============================] - 55s 5s/step - loss: 0.4415 - acc: 0.7917 - val_loss: 0.4193 - val_acc: 0.7950
Epoch 11/30
12/12 [==============================] - 55s 5s/step - loss: 0.4699 - acc: 0.7792 - val_loss: 0.3594 - val_acc: 0.8850
Epoch 12/30
12/12 [==============================] - 52s 4s/step - loss: 0.4257 - acc: 0.7919 - val_loss: 0.3487 - val_acc: 0.8900
Epoch 13/30
12/12 [==============================] - 55s 5s/step - loss: 0.3435 - acc: 0.8458 - val_loss: 0.3224 - val_acc: 0.8500
Epoch 14/30
12/12 [==============================] - 53s 4s/step - loss: 0.4473 - acc: 0.8125 - val_loss: 0.3347 - val_acc: 0.8400
Epoch 15/30
12/12 [==============================] - 54s 5s/step - loss: 0.3332 - acc: 0.8792 - val_loss: 0.3555 - val_acc: 0.8200
Epoch 16/30
12/12 [==============================] - 54s 4s/step - loss: 0.3619 - acc: 0.8184 - val_loss: 0.3004 - val_acc: 0.8750
Epoch 17/30
12/12 [==============================] - 62s 5s/step - loss: 0.3533 - acc: 0.8542 - val_loss: 0.3024 - val_acc: 0.8800
Epoch 18/30
12/12 [==============================] - 69s 6s/step - loss: 0.3745 - acc: 0.8375 - val_loss: 0.5220 - val_acc: 0.7700
Epoch 19/30
12/12 [==============================] - 62s 5s/step - loss: 0.2999 - acc: 0.8833 - val_loss: 0.2733 - val_acc: 0.9000
Epoch 20/30
12/12 [==============================] - 63s 5s/step - loss: 0.4158 - acc: 0.8184 - val_loss: 0.2860 - val_acc: 0.8750
Epoch 21/30
12/12 [==============================] - 59s 5s/step - loss: 0.2793 - acc: 0.8750 - val_loss: 0.2614 - val_acc: 0.8900
Epoch 22/30
12/12 [==============================] - 57s 5s/step - loss: 0.2571 - acc: 0.8917 - val_loss: 0.3202 - val_acc: 0.8450
Epoch 23/30
12/12 [==============================] - 59s 5s/step - loss: 0.2535 - acc: 0.9078 - val_loss: 0.2524 - val_acc: 0.8950
Epoch 24/30
12/12 [==============================] - 58s 5s/step - loss: 0.2694 - acc: 0.9083 - val_loss: 0.2847 - val_acc: 0.8700
Epoch 25/30
12/12 [==============================] - 61s 5s/step - loss: 0.2539 - acc: 0.9083 - val_loss: 1.0310 - val_acc: 0.6750
Epoch 26/30
12/12 [==============================] - 54s 5s/step - loss: 0.2804 - acc: 0.9000 - val_loss: 0.2384 - val_acc: 0.8950
Epoch 27/30
12/12 [==============================] - 58s 5s/step - loss: 0.1992 - acc: 0.9120 - val_loss: 0.4206 - val_acc: 0.8100
Epoch 28/30
12/12 [==============================] - 54s 4s/step - loss: 0.2246 - acc: 0.9292 - val_loss: 0.2523 - val_acc: 0.9100
Epoch 29/30
12/12 [==============================] - 54s 4s/step - loss: 0.1914 - acc: 0.9292 - val_loss: 0.3012 - val_acc: 0.8750
Epoch 30/30
12/12 [==============================] - 53s 4s/step - loss: 0.2023 - acc: 0.9167 - val_loss: 0.2651 - val_acc: 0.9200"""